# Advanced features in McStasScript
Now that you have mastered reading and writing McStasScript instruments it is time to look at some of the more advanced features of McStasScript.

## Use Python functions
Structuring code in functions or even classes can be very helpful.

In [ ]:
import mcstasscript as ms
    
def make_instrument():
    instrument = ms.McStas_instr("demo", input_path="run_folder")
    instrument.settings(output_path="data_folder/talk_2")
    
    src = instrument.add_component("Source", "Source_simple")
    src.set_parameters(xwidth=0.1, yheight=0.1, focus_xw=0.04, focus_yh=0.06, dist=2)
    wavelength = instrument.add_parameter("wavelength", value=3.0, comment="Wavelength center [AA]")
    wavelength_spread = instrument.add_parameter("wavelength_spread", value=2.5, comment="Wavelength spread [AA]")
    src.lambda0 = wavelength
    src.dlambda = wavelength_spread
    
    return instrument

In [ ]:
instrument = make_instrument()

### Use function to add a section

In [ ]:
def add_monitors(instrument, name, xwidth=0.05, yheight=0.05):
    mon_name = name + "_monitor"
    
    mon = instrument.add_component(mon_name, "PSD_monitor")
    mon.set_parameters(xwidth=xwidth, yheight=yheight, restore_neutron=1,
                       filename='"' + mon_name + '.dat"')
    mon.set_AT(0.1, RELATIVE="PREVIOUS")

In [ ]:
add_monitors(instrument, "source", xwidth=0.12, yheight=0.12)
instrument.show_components()

## Use for loops
Simple for loops can save a lot of work, for example here with a curved guide.

In [ ]:
source = instrument.get_component("Source")
last_element = source
last_length = source.dist

for index in range(8):
    name = "guide_" + str(index)
    guide = instrument.add_component(name, "Guide_gravity")
    guide.set_parameters(l=0.5, m=3, w1=source.focus_xw, h1=source.focus_yh)
    guide.set_AT(last_length, RELATIVE=last_element)
    guide.set_ROTATED([0, 0.5, 0], RELATIVE=last_element)
    
    last_element = guide
    last_length = guide.l + 0.005

In [ ]:
guide_end = instrument.add_component("guide_end", "Arm")
guide_end.set_AT(guide.l, RELATIVE=last_element)

add_monitors(instrument, "after_guide", xwidth=0.06, yheight=0.08)

### Show the instrument

In [ ]:
instrument.show_instrument()

## Reader
McStasScript has a system for reading a typical McStas instrument file and returning either a Python instrument object or even writing the python file. This system is not perfect, there are some features that can confuse it.

In [ ]:
# Create a reader object from file path
instrument_reader = ms.McStas_file("PSI_DMC.instr")

# Make a clean instrument object
instrument = ms.McStas_instr("DMC_read", input_path="run_folder")
instrument.settings(output_path="data_folder/read_PSI_DMC")

# The reader can add the information to an instrument object
instrument_reader.add_to_instr(instrument)

instrument.show_parameters()

### Show read instrument geometry

In [ ]:
instrument.show_instrument()

### Write Python file
The written Python file is human readable, but does not quite follow the best practice. 

Can and will be improved in the future!

In [ ]:
instrument_reader.write_python_file("PSI_DMC.py", force=True)

## Analysis mode of diagram

In [ ]:
instrument.show_diagram(analysis=True)

In [ ]:
source = instrument.get_component("source")
source.Lmin = instrument.add_parameter("Lmin", value=1.0)
source.Lmax = instrument.add_parameter("Lmax", value=3.0)
instrument.set_parameters(Lmin=0.5, wavelength=3, Lmax=4)

In [ ]:
instrument.show_diagram(variable="l", limits=[0.5, 4])

### Replace monochromator
It seems the used monochromator component ignores higher order scattering, lets replace it.

In [ ]:
instrument.get_component("foc_mono")

In [ ]:
instrument.remove_component("foc_mono")
mono = instrument.add_component("foc_mono", "Monochromator_curved", after="sma")
mono.set_parameters(zwidth=0.05, yheight=0.025, gap=0.0005,
                    NH=1, NV=5, mosaich=38, mosaicv=38, r0=0.7,
                    Q="mono_q", RV="RV", RH=0)
mono.set_AT(0, RELATIVE="sma")

### See the analysis with new monochromator component

In [ ]:
instrument.set_parameters(Lmin=0.5, wavelength=3, Lmax=4)
instrument.show_diagram(variable="l", limits=[0.5, 4])

# Available variables
The *variable* keyword can take the following variables:

| Axis string | Full name | Unit | Description |
| --- | --- | --- | --- |
| t | time | s | Particle time |
| x | x position | m | Coordinate x of particle |
| y | y position | m | Coordinate y of particle |
| z | z position | m | Coordinate z of particle |
| vx | x velocity | m/s | Velocity projected onto x |
| vy | y velocity | m/s | Velocity projected onto y |
| vz | z velocity | m/s | Velocity projected onto z |
| l | lambda | AA | Wavelength |
| e | energy | meV | Particle energy |
| speed | speed | m/s | Speed (length of velocity vector) |
| dx | x divergence | deg | Divergence from z axis along x axis |
| dy | y divergence | deg | Divergence from z axis along y axis |

In [ ]:
instrument.set_parameters(Lmin=3.0, wavelength=3.1, Lmax=3.2)
instrument.show_diagram(variable="t")

## Diagnostics tool
McStas instruments usually contain a large number of monitors. Usually only really needed when writing the instrument as debugging tool, but also often left in place in case they are needed again.

McStasScript has an different way of handling diagnostics!

In [ ]:
diag = ms.Diagnostics(instrument)
diag.settings(suppress_output=True, ncount=5E6)
diag.set_parameters(Lmin=2.8, wavelength=3, Lmax=3.2)

diag.add_point(before="guide1")
diag.add_point(after="window1")
diag.add_point(after="out1_slit")
diag.run()

In [ ]:
print(diag)

### Views with diagnostics tool
After a diagnostics run is performed, one can investigate different views by adding these.

In [ ]:
diag.clear_views()
diag.add_view("l")
diag.add_view("l", "dx")
diag.add_view("x", "dx")
diag.plot()

## MCPL bridges
McStas can dump the the beam to a MCPL file and pick up such a dump to continue in another instrument. This can be beneficial in several cases:
- One part of the instrument is slow to run
- Too many McStas components in one instrument file

It takes some effort to do this, and especially to keep track of the dump files, but McStasScript has systems to make this easy!

### Segmenting the DMC instrument

In [ ]:
instrument.show_diagram()

### Segment at window2
Lets segment at the *window2* component, as no component after it refers to something before *window2*.

In [ ]:
instrument.run_to("window2")
instrument.show_run_subset()

In [ ]:
instrument.show_diagram()

### Creating a MCPL file
Running the instrument in this state will still provide the usual data from monitors, but also the MCPL file describing the beam at that point.

In [ ]:
instrument.settings(suppress_output=True)
instrument.set_parameters(Lmin=0.8, wavelength=3, Lmax=4.0)
data = instrument.backengine()
print(data)

The instrument object keeps track of all the saved MCPL data, it can be shown with *show_dumps* and even persists when reloading a notebook as the database is written to disk and saved with the instrument name.

In [ ]:
instrument.show_dumps()

### Specifying run names
One can specify a run name and insert a comment to make it easier to remember what the runs mean.

In [ ]:
instrument.set_parameters(Lmin=2.7, wavelength=2.9, Lmax=3.1)
instrument.run_to("window2", run_name="narrow_band", comment="Demo run")
data = instrument.backengine()
instrument.show_dumps()

One can view more details on a specific dump using *show_dump*

In [ ]:
instrument.show_dump("window2", run_name="narrow_band", tag=0)

## Running from a beam dump

In [ ]:
instrument.reset_run_points()
instrument.run_from("window2")
instrument.show_run_subset()

It is possible to adjust what run to load including tag and to set parameters for the *MCPL_output* component used instead of window2.

In [ ]:
instrument.run_from("window2", run_name="narrow_band", tag=0,
                    repeat_count=2, E_smear=0.01)

## Check the instrument diagram
The instrument diagram now show the rest of the instrument.

In [ ]:
instrument.show_diagram()

## Run instrument and plot data

In [ ]:
instrument.set_parameters(wavelength=3)
data = instrument.backengine()
ms.make_sub_plot(data)

# Summary
McStasScript contains several powerful advanced features
- Instrument reader for classic instrument files
- Analysis mode for diagrams
- Diagnostics tool
- MCPL bridge with database

We have also looked at how convenient it is to use functions and loops!

# Exercise 2
You now know about many more McStasScript features and are ready for the next exercises!

The next exercise sections is split into 3 notebooks, each concerning a mystery instrument object you should investigate using the McStasScript help features.